# Step 3: CNN Dataset Generation

## 🎯 Overview

This notebook demonstrates the CNN dataset generation process. We combine ESM2 embeddings with template-derived features to create multi-channel input tensors for binary contact prediction.

## 📋 Learning Objectives

- Understand multi-channel input tensor creation
- Integrate ESM2 embeddings with template features
- Generate synchronized CNN training datasets
- Validate dataset quality and consistency

## 🔬 Key Concepts

**Multi-channel Input**: 68-channel tensors combining sequence and structural features.

**ESM2 Embeddings**: 64 channels of learned protein sequence representations.

**Template Features**: 4 channels of structural information from homologous templates.

**Binary Targets**: 0/1 contact maps for supervised learning.

---

## 🛠️ Setup and Dependencies

In [ ]:
# Import required libraries
import sys
import numpy as np
import h5py
import matplotlib.pyplot as plt
from pathlib import Path
import json
from typing import Dict, List, Tuple, Optional
import warnings
warnings.filterwarnings('ignore')

# Add project root to path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

# Import project modules
from src.esm2_contact.dataset.processing import CNNDataGenerator
from src.esm2_contact.training.dataset import Tiny10Dataset

print("✅ Dependencies imported successfully")
print(f"📁 Project root: {project_root}")
print(f"🐍 Python version: {sys.version.split()[0]}")

## 📊 Prerequisites Check

In [ ]:
def check_prerequisites():
    """
    Check that all prerequisite data files exist.
    """
    print("🔍 Checking Prerequisites")
    print("="*30)
    
    data_dir = project_root / "data" / "tiny_10"
    
    required_files = {
        "ground_truth.h5": "Binary contact maps from Step 1",
        "sequences.json": "Protein sequences", 
        "homology_results.json": "Template search results from Step 2"
    }
    
    all_exist = True
    
    for filename, description in required_files.items():
        filepath = data_dir / filename
        exists = filepath.exists()
        status = "✅" if exists else "❌"
        print(f"   {status} {filename}: {description}")
        
        if exists:
            size_mb = filepath.stat().st_size / (1024 * 1024)
            print(f"      Size: {size_mb:.1f} MB")
        
        all_exist = all_exist and exists
    
    # Check for ESM2 embeddings
    embeddings_dir = project_root / "data" / "esm2_embeddings"
    embeddings_exist = embeddings_dir.exists() and len(list(embeddings_dir.glob("*.npy"))) > 0
    print(f"   {'✅' if embeddings_exist else '❌'} ESM2 embeddings: Language model features")
    
    if embeddings_exist:
        embedding_files = list(embeddings_dir.glob("*.npy"))
        print(f"      Found {len(embedding_files)} embedding files")
    
    all_exist = all_exist and embeddings_exist
    
    print(f"\n📊 Prerequisite Status: {'✅ Complete' if all_exist else '❌ Missing files'}")
    
    if not all_exist:
        print("\n⚠️  Missing prerequisites:")
        if not (data_dir / "ground_truth.h5").exists():
            print("   • Run Step 1: uv run python scripts/create_10_ground_truth_fast.py")
        if not (data_dir / "homology_results.json").exists():
            print("   • Run Step 2: uv run python scripts/run_10_homology.py")
        if not embeddings_exist:
            print("   • Generate ESM2 embeddings using compute_esm2_embeddings.py")
    
    return all_exist

# Check prerequisites
prereqs_ok = check_prerequisites()

## 🧠 Understanding Multi-channel Input Architecture

In [ ]:
def explain_input_architecture():
    """
    Explain the 68-channel input architecture for the CNN.
    """
    print("🧠 CNN Input Architecture Explanation")
    print("="*45)
    
    print("\n📊 Multi-channel Input Structure:")
    print("   Total channels: 68")
    print("   Input shape: (68, L, L) where L = sequence length")
    
    print("\n🔬 Channel Breakdown:")
    
    # Template channels (4)
    template_channels = [
        "Template Distance Map",
        "Template Contact Map", 
        "Template Coverage Map",
        "Template Confidence Map"
    ]
    
    print("\n   🏗️  Template Channels (4):")
    for i, channel in enumerate(template_channels, 1):
        print(f"      Channel {i}: {channel}")
    
    # ESM2 embedding channels (64)
    print("\n   🧬 ESM2 Embedding Channels (64):")
    print("      Channels 5-68: Outer product of ESM2 embeddings")
    print("      Captures sequence-residue relationships")
    print("      Provides evolutionary and structural context")
    
    print("\n🔄 Data Integration Process:")
    print("   1. Load ESM2 embeddings (L × 64 matrix)")
    print("   2. Compute outer product: (L × 64) × (64 × L) = L × L × 64")
    print("   3. Process template features into 4 L × L matrices")
    print("   4. Stack all channels: 64 + 4 = 68 channels total")
    print("   5. Synchronize with ground truth contact maps")
    
    print("\n🎯 Benefits of Multi-channel Approach:")
    print("   • Combines sequence information (ESM2) with structural templates")
    print("   • Provides rich feature representation for CNN learning")
    print("   • Enables model to leverage both homology and language model cues")
    print("   • Supports Strategy 1: Homology-Assisted CNN approach")
    
    # Create a visual representation
    print("\n📈 Visual Representation:")
    print("   ┌─────────────────────────────────────┐")
    print("   │           Input Tensor (68, L, L)      │")
    print("   ├─────────────────────────────────────┤")
    print("   │ Template Channels 1-4 (4 × L × L)     │")
    print("   ├─────────────────────────────────────┤")
    print("   │ ESM2 Outer Product (64 × L × L)       │")
    print("   └─────────────────────────────────────┘")
    print("                    ↓")
    print("   ┌─────────────────────────────────────┐")
    print("   │            Binary CNN Model           │")
    print("   └─────────────────────────────────────┘")
                    ↓
    print("   ┌─────────────────────────────────────┐")
    print("   │         Contact Map (L × L)           │")
    print("   └─────────────────────────────────────┘")

# Explain architecture
explain_input_architecture()

## 📂 Loading and Analyzing Input Data

In [ ]:
def load_and_analyze_input_data():
    """
    Load and analyze all input data components for CNN dataset generation.
    """
    if not prereqs_ok:
        print("❌ Prerequisites not met")
        return None
    
    print("📂 Loading Input Data Components")
    print("="*40)
    
    data_dir = project_root / "data" / "tiny_10"
    
    # Load ground truth data
    print("\n🎯 Loading Ground Truth Data...")
    with h5py.File(data_dir / "ground_truth.h5", 'r') as f:
        ground_truth = {}
        for protein_id in f.keys():
            ground_truth[protein_id] = {
                'contact_map': f[protein_id]['contact_map'][:],
                'sequence': f[protein_id]['sequence'][:].decode('utf-8')
            }
    
    print(f"   Loaded {len(ground_truth)} proteins with ground truth")
    
    # Load homology results
    print("\n🔍 Loading Homology Results...")
    with open(data_dir / "homology_results.json", 'r') as f:
        homology_results = json.load(f)
    
    query_results = homology_results.get('results', {})
    print(f"   Loaded homology results for {len(query_results)} queries")
    
    # Check ESM2 embeddings
    print("\n🧠 Checking ESM2 Embeddings...")
    embeddings_dir = project_root / "data" / "esm2_embeddings"
    embedding_files = list(embeddings_dir.glob("*.npy"))
    
    print(f"   Found {len(embedding_files)} embedding files")
    
    # Analyze data consistency
    print("\n📊 Data Consistency Analysis:")
    
    common_proteins = set(ground_truth.keys()) & set(query_results.keys())
    print(f"   Proteins with ground truth: {len(ground_truth)}")
    print(f"   Proteins with homology results: {len(query_results)}")
    print(f"   Common proteins: {len(common_proteins)}")
    
    # Check protein with templates
    proteins_with_templates = [pid for pid, result in query_results.items() 
                               if result.get('templates')]
    print(f"   Proteins with templates: {len(proteins_with_templates)}")
    
    # Sample protein analysis
    if common_proteins:
        sample_protein = list(common_proteins)[0]
        print(f"\n📋 Sample Analysis: {sample_protein}")
        
        gt_data = ground_truth[sample_protein]
        holo_data = query_results[sample_protein]
        
        print(f"   Sequence length: {len(gt_data['sequence'])}")
        print(f"   Contact map shape: {gt_data['contact_map'].shape}")
        print(f"   Contact density: {np.mean(gt_data['contact_map']):.4f}")
        print(f"   Templates found: {len(holo_data.get('templates', []))}")
        
        # Check for embedding file
        embedding_file = embeddings_dir / f"{sample_protein}.npy"
        if embedding_file.exists():
            embedding = np.load(embedding_file)
            print(f"   ESM2 embedding shape: {embedding.shape}")
        else:
            print(f"   ESM2 embedding: Missing")
    
    return {
        'ground_truth': ground_truth,
        'homology_results': query_results,
        'common_proteins': common_proteins,
        'proteins_with_templates': proteins_with_templates
    }

# Load input data
input_data = load_and_analyze_input_data()

## 🔬 CNN Data Generation Process

In [ ]:
def demonstrate_cnn_data_generation():
    """
    Demonstrate the CNN data generation process.
    """
    if not input_data:
        print("❌ No input data available")
        return None
    
    print("🔬 CNN Data Generation Demonstration")
    print("="*45)
    
    # Initialize data generator
    data_dir = project_root / "data" / "tiny_10"
    embeddings_dir = project_root / "data" / "esm2_embeddings"
    
    print("\n⚙️ Data Generator Configuration:")
    print(f"   Ground truth file: {data_dir / 'ground_truth.h5'}")
    print(f"   Homology results: {data_dir / 'homology_results.json'}")
    print(f"   ESM2 embeddings: {embeddings_dir}")
    print(f"   Output file: {data_dir / 'cnn_dataset.h5'}")
    
    # Sample the generation process for one protein
    common_proteins = input_data['common_proteins']
    proteins_with_templates = input_data['proteins_with_templates']
    
    # Find a protein with all required data
    sample_protein = None
    for protein in common_proteins:
        if protein in proteins_with_templates:
            embedding_file = embeddings_dir / f"{protein}.npy"
            if embedding_file.exists():
                sample_protein = protein
                break
    
    if not sample_protein:
        print("❌ No protein found with all required data")
        return None
    
    print(f"\n🧬 Processing Sample Protein: {sample_protein}")
    
    # Load data for sample protein
    gt_data = input_data['ground_truth'][sample_protein]
    holo_data = input_data['homology_results'][sample_protein]
    embedding = np.load(embeddings_dir / f"{sample_protein}.npy")
    
    sequence = gt_data['sequence']
    contact_map = gt_data['contact_map']
    templates = holo_data.get('templates', [])
    
    print(f"   Sequence length: {len(sequence)}")
    print(f"   Contact map shape: {contact_map.shape}")
    print(f"   ESM2 embedding shape: {embedding.shape}")
    print(f"   Available templates: {len(templates)}")
    
    # Demonstrate outer product computation
    print("\n🔄 ESM2 Outer Product Computation:")
    print(f"   Input shape: {embedding.shape}")
    
    # Compute outer product (this is what the CNN data generator does)
    outer_product = np.outer(embedding, embedding)
    print(f"   Outer product shape: {outer_product.shape}")
    
    # Reshape to (64, L, L) format
    L = len(sequence)
    esm2_channels = outer_product.reshape(64, L, L)
    print(f"   ESM2 channels shape: {esm2_channels.shape}")
    
    print("\n🏗️ Template Feature Processing:")
    if templates:
        best_template = templates[0]
        print(f"   Best template: {best_template.get('template_pdb', 'Unknown')}")
        print(f"   Template identity: {best_template.get('sequence_identity', 0):.3f}")
        print(f"   Template coverage: {best_template.get('coverage', 0):.3f}")
        
        # Create synthetic template features for demonstration
        # (In real process, these are computed from actual template structures)
        template_distance_map = np.random.rand(L, L) * 20  # Random distances
        template_contact_map = (template_distance_map < 8).astype(float)  # Binary contacts
        template_coverage_map = np.ones((L, L)) * best_template.get('coverage', 0.5)
        template_confidence_map = np.ones((L, L)) * best_template.get('sequence_identity', 0.5)
        
        template_channels = np.stack([
            template_distance_map,
            template_contact_map,
            template_coverage_map,
            template_confidence_map
        ], axis=0)
        
        print(f"   Template channels shape: {template_channels.shape}")
    else:
        # Create dummy template features
        template_channels = np.zeros((4, L, L))
        print(f"   No templates found, using dummy features: {template_channels.shape}")
    
    # Combine all channels
    print("\n🔗 Channel Combination:")
    multi_channel_input = np.concatenate([template_channels, esm2_channels], axis=0)
    print(f"   Final input shape: {multi_channel_input.shape}")
    print(f"   Expected channels: 68 (4 template + 64 ESM2)")
    
    # Validate channel dimensions
    assert multi_channel_input.shape[0] == 68, f"Expected 68 channels, got {multi_channel_input.shape[0]}"
    assert multi_channel_input.shape[1] == multi_channel_input.shape[2] == L, "Matrix should be square"
    
    print("   ✅ Channel dimensions validated")
    
    # Data quality checks
    print("\n🔍 Data Quality Checks:")
    print(f"   Input contains NaN: {np.isnan(multi_channel_input).any()}")
    print(f"   Input contains Inf: {np.isinf(multi_channel_input).any()}")
    print(f"   Input range: [{multi_channel_input.min():.4f}, {multi_channel_input.max():.4f}]")
    print(f"   Contact map density: {np.mean(contact_map):.4f}")
    print(f"   Contact map range: [{contact_map.min():.4f}, {contact_map.max():.4f}]")
    
    return {
        'sample_protein': sample_protein,
        'multi_channel_input': multi_channel_input,
        'contact_map': contact_map,
        'sequence': sequence
    }

# Demonstrate CNN data generation
cnn_demo = demonstrate_cnn_data_generation()

## 📊 Multi-channel Input Visualization

In [ ]:
def visualize_multi_channel_input():
    """
    Visualize the multi-channel input structure.
    """
    if not cnn_demo:
        print("❌ No CNN demo data to visualize")
        return
    
    print("📊 Multi-channel Input Visualization")
    print("="*45)
    
    multi_channel_input = cnn_demo['multi_channel_input']
    contact_map = cnn_demo['contact_map']
    sequence = cnn_demo['sequence']
    sample_protein = cnn_demo['sample_protein']
    
    L = len(sequence)
    
    # Create comprehensive visualization
    fig, axes = plt.subplots(3, 3, figsize=(15, 12))
    
    # Template channels (4) - show first 3
    for i in range(3):
        im = axes[0, i].imshow(multi_channel_input[i], cmap='viridis', origin='lower')
        axes[0, i].set_title(f'Template Channel {i+1}')
        axes[0, i].set_xlabel('Residue Index')
        if i == 0:
            axes[0, i].set_ylabel('Residue Index')
        plt.colorbar(im, ax=axes[0, i], fraction=0.046, pad=0.04)
    
    # ESM2 channels - show sample channels
    esm2_channel_indices = [4, 10, 20]  # Sample from ESM2 channels
    for i, channel_idx in enumerate(esm2_channel_indices):
        im = axes[1, i].imshow(multi_channel_input[channel_idx], cmap='RdBu_r', origin='lower')
        axes[1, i].set_title(f'ESM2 Channel {channel_idx+1}')
        axes[1, i].set_xlabel('Residue Index')
        if i == 0:
            axes[1, i].set_ylabel('Residue Index')
        plt.colorbar(im, ax=axes[1, i], fraction=0.046, pad=0.04)
    
    # Target and statistics
    # Ground truth contact map
    im = axes[2, 0].imshow(contact_map, cmap='binary', origin='lower')
    axes[2, 0].set_title('Ground Truth Contact Map')
    axes[2, 0].set_xlabel('Residue Index')
    axes[2, 0].set_ylabel('Residue Index')
    plt.colorbar(im, ax=axes[2, 0], fraction=0.046, pad=0.04)
    
    # Channel statistics
    channel_means = np.mean(multi_channel_input, axis=(1, 2))
    channel_stds = np.std(multi_channel_input, axis=(1, 2))
    
    axes[2, 1].plot(channel_means, label='Mean', alpha=0.7)
    axes[2, 1].axvline(x=4, color='red', linestyle='--', label='ESM2 start')
    axes[2, 1].set_xlabel('Channel Index')
    axes[2, 1].set_ylabel('Mean Value')
    axes[2, 1].set_title('Channel Means')
    axes[2, 1].legend()
    axes[2, 1].grid(True, alpha=0.3)
    
    axes[2, 2].plot(channel_stds, label='Std Dev', alpha=0.7, color='orange')
    axes[2, 2].axvline(x=4, color='red', linestyle='--', label='ESM2 start')
    axes[2, 2].set_xlabel('Channel Index')
    axes[2, 2].set_ylabel('Standard Deviation')
    axes[2, 2].set_title('Channel Standard Deviations')
    axes[2, 2].legend()
    axes[2, 2].grid(True, alpha=0.3)
    
    plt.suptitle(f'Multi-channel CNN Input Analysis - {sample_protein} (L={L})', 
                 fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # Print detailed statistics
    print(f"\n📈 Detailed Channel Statistics:")
    print(f"   Protein: {sample_protein}")
    print(f"   Sequence length: {L}")
    print(f"   Input shape: {multi_channel_input.shape}")
    
    print(f"\n   Template Channels (1-4):")
    for i in range(4):
        channel_data = multi_channel_input[i]
        print(f"      Channel {i+1}: mean={np.mean(channel_data):.4f}, "
              f"std={np.std(channel_data):.4f}, "
              f"range=[{np.min(channel_data):.4f}, {np.max(channel_data):.4f}]")
    
    print(f"\n   ESM2 Channels (5-68) - Summary:")
    esm2_data = multi_channel_input[4:]  # Channels 5-68
    print(f"      Mean: {np.mean(esm2_data):.4f}")
    print(f"      Std: {np.std(esm2_data):.4f}")
    print(f"      Range: [{np.min(esm2_data):.4f}, {np.max(esm2_data):.4f}]")
    print(f"      Negative values: {np.sum(esm2_data < 0)} ({np.mean(esm2_data < 0)*100:.1f}%)")
    print(f"      Positive values: {np.sum(esm2_data > 0)} ({np.mean(esm2_data > 0)*100:.1f}%)")
    
    print(f"\n   Target Contact Map:")
    print(f"      Density: {np.mean(contact_map):.4f}")
    print(f"      Total contacts: {np.sum(contact_map)}")
    print(f"      Contact percentage: {np.mean(contact_map)*100:.2f}%")

# Visualize multi-channel input
visualize_multi_channel_input()

## 🚀 Running CNN Dataset Generation

In [ ]:
def run_cnn_dataset_generation():
    """
    Execute the complete CNN dataset generation pipeline.
    """
    print("🚀 Running CNN Dataset Generation Pipeline")
    print("="*50)
    
    import subprocess
    import os
    
    # Set environment variables
    env = os.environ.copy()
    env['PYTHONPATH'] = str(project_root)
    
    # Run the CNN dataset generation script
    script_path = project_root / "scripts" / "generate_10_cnn_dataset.py"
    
    if not script_path.exists():
        print(f"❌ Script not found: {script_path}")
        return False
    
    print(f"📂 Script: {script_path}")
    print(f"📁 Working directory: {project_root}")
    print("⏱️  This process may take 5-10 minutes...")
    
    try:
        # Run the CNN dataset generation script
        result = subprocess.run(
            ["uv", "run", "python", str(script_path)],
            cwd=project_root,
            capture_output=True,
            text=True,
            env=env,
            timeout=600  # 10 minutes timeout
        )
        
        print("📊 Pipeline Output:")
        print(result.stdout)
        
        if result.stderr:
            print("\n⚠️ Warnings/Errors:")
            print(result.stderr)
        
        if result.returncode == 0:
            print("\n✅ CNN dataset generation completed successfully!")
            
            # Verify output files
            data_dir = project_root / "data" / "tiny_10"
            cnn_dataset_file = data_dir / "cnn_dataset.h5"
            
            if cnn_dataset_file.exists():
                file_size_mb = cnn_dataset_file.stat().st_size / (1024 * 1024)
                print(f"\n📁 Generated files:")
                print(f"   ✅ {cnn_dataset_file} ({file_size_mb:.1f} MB)")
                
                # Quick validation
                with h5py.File(cnn_dataset_file, 'r') as f:
                    if 'cnn_data' in f:
                        protein_count = len(f['cnn_data'].keys())
                        print(f"   📊 Processed {protein_count} proteins")
                        
                        # Check first protein
                        sample_id = list(f['cnn_data'].keys())[0]
                        sample_group = f['cnn_data'][sample_id]
                        
                        if 'multi_channel_input' in sample_group:
                            input_shape = sample_group['multi_channel_input'].shape
                            print(f"   📏 Sample input shape: {input_shape}")
                            
                            if 'consensus_contact_map' in sample_group:
                                contact_shape = sample_group['consensus_contact_map'].shape
                                print(f"   📏 Sample contact shape: {contact_shape}")
                        
                        return True
                    else:
                        print("   ❌ Invalid dataset format: missing 'cnn_data' group")
                        return False
            else:
                print("❌ Expected output file not found")
                return False
        else:
            print(f"❌ Pipeline failed with return code: {result.returncode}")
            return False
            
    except subprocess.TimeoutExpired:
        print("❌ Pipeline timed out after 10 minutes")
        return False
    except Exception as e:
        print(f"❌ Error running pipeline: {e}")
        return False

# Uncomment to run the CNN dataset generation pipeline
# success = run_cnn_dataset_generation()
# print(f"\n🎯 Pipeline {'succeeded' if success else 'failed'}!")

print("📝 CNN dataset generation pipeline ready.")
print("💡 Uncomment the function call above to execute the generation.")
print("⚠️  This will process all 10 proteins and may take 5-10 minutes.")

## 📊 Analyze Generated CNN Dataset

In [ ]:
def analyze_generated_cnn_dataset():
    """
    Analyze the generated CNN dataset for quality and consistency.
    """
    data_dir = project_root / "data" / "tiny_10"
    cnn_dataset_file = data_dir / "cnn_dataset.h5"
    
    if not cnn_dataset_file.exists():
        print("❌ No CNN dataset found. Run the generation pipeline first.")
        return None
    
    print("📊 Analyzing Generated CNN Dataset")
    print("="*40)
    
    with h5py.File(cnn_dataset_file, 'r') as f:
        if 'cnn_data' not in f:
            print("❌ Invalid dataset format: missing 'cnn_data' group")
            return None
        
        cnn_data = f['cnn_data']
        protein_ids = list(cnn_data.keys())
        
        print(f"\n📋 Dataset Overview:")
        print(f"   Total proteins: {len(protein_ids)}")
        print(f"   File size: {cnn_dataset_file.stat().st_size / (1024*1024):.1f} MB")
        
        # Analyze each protein
        all_lengths = []
        all_densities = []
        all_input_shapes = []
        all_contact_shapes = []
        
        print(f"\n🔍 Protein-wise Analysis:")
        
        for i, protein_id in enumerate(protein_ids):
            protein_group = cnn_data[protein_id]
            
            # Extract data
            multi_channel_input = protein_group['multi_channel_input'][:]
            contact_map = protein_group['consensus_contact_map'][:]
            seq_len = protein_group.attrs.get('sequence_length', contact_map.shape[0])
            
            # Validate shapes
            input_shape = multi_channel_input.shape
            contact_shape = contact_map.shape
            
            # Calculate statistics
            density = np.mean(contact_map)
            
            all_lengths.append(seq_len)
            all_densities.append(density)
            all_input_shapes.append(input_shape)
            all_contact_shapes.append(contact_shape)
            
            print(f"   {i+1:2d}. {protein_id}:")
            print(f"       Length: {seq_len}")
            print(f"       Input shape: {input_shape}")
            print(f"       Contact shape: {contact_shape}")
            print(f"       Contact density: {density:.4f}")
            
            # Data quality checks
            has_nan = np.isnan(multi_channel_input).any()
            has_inf = np.isinf(multi_channel_input).any()
            binary_contacts = np.all(np.isin(contact_map, [0.0, 1.0]))
            
            quality_issues = []
            if has_nan:
                quality_issues.append("NaN values")
            if has_inf:
                quality_issues.append("Inf values")
            if not binary_contacts:
                quality_issues.append("Non-binary contacts")
            if input_shape[0] != 68:
                quality_issues.append(f"Wrong channel count: {input_shape[0]}")
            
            if quality_issues:
                print(f"       ⚠️  Issues: {', '.join(quality_issues)}")
            else:
                print(f"       ✅ Quality: OK")
            print()
        
        # Summary statistics
        print(f"📈 Dataset Statistics:")
        print(f"   Length range: {min(all_lengths)} - {max(all_lengths)} residues")
        print(f"   Average length: {np.mean(all_lengths):.1f} residues")
        print(f"   Density range: {min(all_densities):.4f} - {max(all_densities):.4f}")
        print(f"   Average density: {np.mean(all_densities):.4f}")
        print(f"   Total contacts: {sum(np.sum(cnn_data[pid]['consensus_contact_map'][:]) for pid in protein_ids):,}")
        
        # Channel statistics
        print(f"\n🧠 Channel Analysis:")
        sample_input = cnn_data[protein_ids[0]]['multi_channel_input'][:]
        
        template_channels = sample_input[:4]  # First 4 channels
        esm2_channels = sample_input[4:]      # Remaining 64 channels
        
        print(f"   Template channels (1-4):")
        for i in range(4):
            channel_data = template_channels[i]
            print(f"      Channel {i+1}: mean={np.mean(channel_data):.4f}, "
                  f"std={np.std(channel_data):.4f}")
        
        print(f"   ESM2 channels (5-68):")
        print(f"      Mean: {np.mean(esm2_channels):.4f}")
        print(f"      Std: {np.std(esm2_channels):.4f}")
        print(f"      Range: [{np.min(esm2_channels):.4f}, {np.max(esm2_channels):.4f}]")
        
        # Cross-dataset consistency
        input_shapes_consistent = all(shape == all_input_shapes[0] for shape in all_input_shapes)
        print(f"\n🔍 Consistency Checks:")
        print(f"   Input shapes consistent: {'✅' if input_shapes_consistent else '❌'}")
        print(f"   All proteins have 68 channels: {'✅' if all(s[0] == 68 for s in all_input_shapes) else '❌'}")
        print(f"   Contact maps are binary: {'✅' if all(np.all(np.isin(cnn_data[pid]['consensus_contact_map'][:], [0.0, 1.0])) for pid in protein_ids) else '❌'}")
        
        return {
            'protein_count': len(protein_ids),
            'protein_ids': protein_ids,
            'lengths': all_lengths,
            'densities': all_densities,
            'input_shapes': all_input_shapes,
            'contact_shapes': all_contact_shapes
        }

# Analyze generated dataset
dataset_analysis = analyze_generated_cnn_dataset()

## 📋 Summary and Key Takeaways

In [ ]:
def summarize_cnn_dataset_generation():
    """
    Summarize the CNN dataset generation process and results.
    """
    print("📋 CNN Dataset Generation Summary")
    print("="*45)
    
    print("\n🎯 Process Overview:")
    print("   1. Load ground truth contact maps (Step 1)")
    print("   2. Load homology search results (Step 2)")
    print("   3. Load ESM2 embeddings (language model features)")
    print("   4. Process template structures into feature maps")
    print("   5. Compute ESM2 outer product (64 channels)")
    print("   6. Combine into 68-channel input tensors")
    print("   7. Synchronize with binary contact map targets")
    
    print("\n🔬 Key Technical Details:")
    print("   • Input format: (68, L, L) tensors per protein")
    print("   • Template channels: 4 (distance, contact, coverage, confidence)")
    print("   • ESM2 channels: 64 (outer product of embeddings)")
    print("   • Target format: Binary L×L contact maps (0/1 values)")
    print("   • Storage: HDF5 format with metadata")
    
    if dataset_analysis:
        print(f"\n📊 Generated Dataset Statistics:")
        print(f"   • Total proteins: {dataset_analysis['protein_count']}")
        print(f"   • Length range: {min(dataset_analysis['lengths'])} - {max(dataset_analysis['lengths'])} residues")
        print(f"   • Average density: {np.mean(dataset_analysis['densities']):.4f}")
        print(f"   • Input channels: 68 (4 template + 64 ESM2)")
        print(f"   • Storage format: HDF5 with synchronized data")
        
        # Quality assessment
        high_density_proteins = sum(1 for d in dataset_analysis['densities'] if d > 0.02)
        low_density_proteins = sum(1 for d in dataset_analysis['densities'] if d < 0.005)
        
        print(f"\n🏆 Dataset Quality Assessment:")
        print(f"   • High density proteins (>2%): {high_density_proteins}")
        print(f"   • Low density proteins (<0.5%): {low_density_proteins}")
        print(f"   • Average density: {np.mean(dataset_analysis['densities']):.4f}")
        print(f"   • Density variance: {np.var(dataset_analysis['densities']):.6f}")
    
    print("\n✅ Quality Assurance Features:")
    print("   • NaN and Inf value detection")
    print("   • Binary contact map validation")
    print("   • Shape consistency checking")
    print("   • Channel dimension verification")
    print("   • Data synchronization validation")
    
    print("\n🔄 Integration with Strategy 1:")
    print("   • Implements Homology-Assisted CNN approach")
    print("   • Combines ESM2 language model features with template data")
    print("   • Provides rich multi-modal input for deep learning")
    print("   • Supports binary contact prediction as required")
    
    print("\n🎯 Dataset Features:")
    print("   • Memory-efficient HDF5 storage")
    print("   • Variable protein length support")
    print("   • Comprehensive metadata inclusion")
    print("   • Ready-to-use for PyTorch DataLoader")
    
    print("\n🎉 Ready for Next Step:")
    print("   CNN dataset is ready for binary CNN training (Step 4)")
    print("   All 68 channels properly formatted and validated")
    print("   Binary contact maps synchronized with inputs")

# Run summary
summarize_cnn_dataset_generation()